In [ ]:
!pip install boto3
!pip install googlemaps

In [ ]:
import sys
import boto3
import googlemaps
import pandas as pd
from geopy.distance import geodesic
import botocore
from tabulate import tabulate
import io


#AWS credentials and bucket name
bucket_name = input ("Bucket name:")
aws_access_key_id = input ("AWS key:")
aws_secret_access_key = input ("AWS secret key:")

# Google API Key
api_key = input ("Google key:")

place_type = input ("Enter place type:")

def check_radius_digit(radius):
    return radius.isdigit()

def check_latitude_longitude_digit(latitude, longitude):
    try:
        float(latitude)
        float(longitude)
        return True
    except ValueError:
        return False

def check_latitude_longitude_range(latitude, longitude):
    try:
        latitude = float(latitude)
        longitude = float(longitude)
        if not (-90 <= latitude <= 90) or not (-180 <= longitude <= 180):
            return False
        return True
    except ValueError:
        return False

def check_positive_number(number):
    return number.replace('.', '').isdigit()

def get_nearby_places(api_key, location, radius, place_type):
    try:
        gmaps = googlemaps.Client(key=api_key)
        places_result = gmaps.places_nearby(location=location, radius=radius, type=place_type)

        if 'results' in places_result:
            return places_result['results']
        else:
            return []
    except googlemaps.exceptions.ApiError as e:
        print("Google Maps API error:", e)
        return []
    except Exception as e:
        print("An error occurred:", e)
        return []

def check_s3_connection():
    try:
        # Connect to S3
        s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

        # List buckets to check if the connection and keys are valid
        response = s3.list_buckets()

        # Check if there are any buckets
        if 'Buckets' not in response:
            print("No buckets found in S3.")
            return None

        # Extract bucket names
        bucket_names = [bucket['Name'] for bucket in response['Buckets']]

        return s3, bucket_names
    except botocore.exceptions.NoCredentialsError:
        print("No AWS credentials found. Please configure your AWS credentials using one of the following methods:")
        print("- Using AWS CLI: aws configure")
        print("- Using environment variables: export AWS_ACCESS_KEY_ID=<YourAccessKeyID>; export AWS_SECRET_ACCESS_KEY=<YourSecretAccessKey>")
        print("- Using IAM roles: Assign an IAM role with appropriate permissions to your AWS resource.")
        return None
    except Exception as e:
        print("An unexpected error occurred while connecting to S3:", e)
        return None

def save_to_s3(data, bucket_name, file_name):
    s3_connection_info = check_s3_connection()
    if s3_connection_info is None:
        return  # Stop execution if connection to S3 failed

    s3, bucket_names = s3_connection_info

    # Check if the specified bucket exists
    if bucket_name not in bucket_names:
        print(f"You cannot save data to the specified bucket '{bucket_name}' does not exist.")
        print("Available buckets:")
        for bucket in bucket_names:
            print(bucket)
        return

    try:
        # Save CSV data to S3
        s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
        csv_data = data.to_csv(index=False)
        s3.put_object(Body=csv_data, Bucket=bucket_name, Key=file_name)

        print("Data saved to S3 successfully.")
        print(f"Bucket: {bucket_name}, File: {file_name}")

        return file_name  # Return the file name saved in S3

    except Exception as e:
        print("An error occurred while saving to S3:", e)
        return None

def show_info_from_s3(bucket_name, file_name):
    if not check_s3_connection():
        return

    try:
        # Retrieve file from S3
        s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
        response = s3.get_object(Bucket=bucket_name, Key=file_name)
        data = response['Body'].read().decode('utf-8')

        # Convert CSV data to DataFrame
        df = pd.read_csv(io.StringIO(data))

        # Display DataFrame as a table
        print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))
    except Exception as e:
        print("An error occurred while retrieving data from S3:", e)

def list_files_in_s3(bucket_name):
    s3_connection_info = check_s3_connection()
    if s3_connection_info is None:
        return  # Stop execution if connection to S3 failed

    s3, bucket_names = s3_connection_info

    # Check if the specified bucket exists
    if bucket_name not in bucket_names:
        print(f"You cannot take list of files from the specified bucket '{bucket_name}' does not exist.")
        print("Available buckets:")
        for bucket in bucket_names:
            print(bucket)
        return

    try:
        # List objects in the bucket
        s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
        response = s3.list_objects_v2(Bucket=bucket_name)

        # Check if the response contains any objects
        if 'Contents' in response:
            print("Files in bucket '{}'".format(bucket_name))
            for obj in response['Contents']:
                print(obj['Key'])
        else:
            print("No files found in bucket '{}'.".format(bucket_name))
    except Exception as e:
        print("An error occurred while listing files in S3:", e)

def delete_rows_from_s3_file(bucket_name, file_name, rows_to_delete):
    if not check_s3_connection():
        return

    try:
        # Retrieve file from S3
        s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
        response = s3.get_object(Bucket=bucket_name, Key=file_name)
        data = response['Body'].read().decode('utf-8')

        # Convert CSV data to DataFrame
        df = pd.read_csv(io.StringIO(data))

        # Delete rows from DataFrame
        for row in rows_to_delete:
            df.drop(row, inplace=True)

        # Convert DataFrame back to CSV
        csv_data = df.to_csv(index=False)

        # Save updated CSV data to S3
        s3.put_object(Body=csv_data, Bucket=bucket_name, Key=file_name)

        print("Rows deleted from S3 file successfully.")

        # Show updated information from file in S3
        show_info_from_s3(bucket_name, file_name)

    except Exception as e:
        print("An error occurred while deleting rows from S3 file:", e)


# Define validate_choice function
def validate_choice(choice):
    return choice in ['1', '2']

def validate_delete_list(delete_list):
    # Valid characters: digits, ',', '-'
    valid_chars = set('0123456789,-')
    return all(char in valid_chars for char in delete_list)

def main():
    saved_file_name = None

    # Input latitude and longitude
    latitude = input("Enter latitude: ")
    longitude = input("Enter longitude: ")

    # Check if latitude and longitude are digits and within valid range
    if not check_latitude_longitude_digit(latitude, longitude) or not check_latitude_longitude_range(latitude, longitude):
        print("Error: Latitude and longitude must be valid decimal numbers between -90 and 90 for latitude, and -180 and 180 for longitude.")
        return

    radius = input("Enter radius in meters: ")

    # Check if radius is a positive number
    if not check_radius_digit(radius) or not check_positive_number(radius):
        print("Error: Radius must be a positive number.")
        return

    location = (float(latitude), float(longitude))
    radius = float(radius)  # Convert radius to meters

    # Define an empty DataFrame
    df = pd.DataFrame(columns=['#', place_type, 'Distance (km)', 'Latitude', 'Longitude'])

    nearby_places = get_nearby_places(api_key, location, radius, place_type)

    if nearby_places:
        data = []
        for idx, place in enumerate(nearby_places, start=1):
            name = place['name']
            latitude = round(place['geometry']['location']['lat'], 6)
            longitude = round(place['geometry']['location']['lng'], 6)
            place_location = (latitude, longitude)
            distance = round(geodesic(location, place_location).kilometers, 3)
            data.append([idx, name, distance, latitude, longitude])

        # Update the DataFrame
        df = pd.DataFrame(data, columns=['#', place_type, 'Distance (km)', 'Latitude', 'Longitude'])

        # Print DataFrame with left-aligned columns
        print(df.to_string(index=False,
                           formatters={'#': '{:<3}'.format, place_type: '{:<50}'.format,
                                       'Distance (km)': '{:<20}'.format, 'Latitude': '{:<20}'.format,
                                       'Longitude': '{:<20}'.format}, justify='left'))
    else:
        # No schools found within the entered radius, check within an extended radius of 200 km
        extended_radius = 200000  # 200 km in meters
        nearby_places_200km = get_nearby_places(api_key, location, extended_radius, place_type)
        if nearby_places_200km:
            data = []
            for idx, place in enumerate(nearby_places_200km, start=1):
                name = place['name']
                latitude = round(place['geometry']['location']['lat'], 6)
                longitude = round(place['geometry']['location']['lng'], 6)
                place_location = (latitude, longitude)
                distance = round(geodesic(location, place_location).kilometers, 3)
                data.append([idx, name, distance, latitude, longitude])
            print("No schools found within the entered radius, {place_type} in a 200 km radius:")
            # Update the DataFrame
            df = pd.DataFrame(data, columns=['#', place_type, 'Distance (km)', 'Latitude', 'Longitude'])

            # Print DataFrame with left-aligned columns
            print(df.to_string(index=False,
                               formatters={'#': '{:<3}'.format, place_type: '{:<50}'.format,
                                           'Distance (km)': '{:<20}'.format, 'Latitude': '{:<20}'.format,
                                           'Longitude': '{:<20}'.format}, justify='left'))
        else:
            print("No {place_type} found within the entered radius and within a 200 km radius.")

    # Save data to S3
    file_name = f'nearby_{place_type}.csv'
    saved_file_name = save_to_s3(df, bucket_name, file_name)

    if saved_file_name:
        # Show information from the file saved in S3
        show_info_from_s3(bucket_name, file_name)

    # Show list of the file saved in S3
    list_files_in_s3(bucket_name)

    if saved_file_name:
        # Ask if user wants to delete rows from the S3 file
        choice = input("Do you want to delete rows from the S3 file? (1 - Yes, 2 - No): ")
        if choice == '2':  # If user chooses 'No', stop the script
            print("Saving file without changes...")
            return  # Stop the script without raising an exception
        elif choice == '1':  # Continue with deleting rows from the S3 file
            delete_rows_loop(bucket_name, file_name)

def delete_rows(bucket_name, file_name):
    while True:
        delete_list = input("Enter the row index(es) you want to delete (e.g., 1,3,5 or 2-4): ")
        if validate_delete_list(delete_list):
            rows_to_delete = []
            for part in delete_list.split(','):
                if '-' in part:
                    start, end = part.split('-')
                    rows_to_delete.extend(range(int(start), int(end)+1))
                else:
                    rows_to_delete.append(int(part))
            delete_rows_from_s3_file(bucket_name, file_name, rows_to_delete)
            break
        else:
            print("Please enter a valid list of row indexes using digits, ',' or '-'.")

def delete_rows_loop(bucket_name, file_name):
    delete_rows(bucket_name, file_name)

    while True:
        delete_more = input("Do you want to delete more rows? (1 - Yes, 2 - No): ")
        if delete_more == '2':
            break
        elif delete_more == '1':
            delete_rows(bucket_name, file_name)
        else:
            print("Invalid choice. Please enter '1' for Yes or '2' for No.")
main()

Enter place type:school
Enter latitude: 52
Enter longitude: 21
Enter radius in meters: 5000
#   school                                                   Distance (km)        Latitude             Longitude           
1         Special Schools                                    2.317                52.020453            21.006321           
2         Kidi. Przedszkole niepubliczne                     2.718                52.002908            21.039303           
3         Pikkolo                                            2.497                52.001462            21.036278           
4         Szkoła Podstawowa im. Mikołaja Kopernika           2.732                52.00318             21.039443           
5         Eureka Elementary School                           3.634                52.030593            21.018511           
6         Małe Przedszkole                                   3.685                51.989553            21.050917           
7         Szkoła Podstawowa             